## Instruction:
<Place holder>

In [2]:
def merge_learning_objects(existing_data, new_data):
    for key in new_data.keys():
        if key in existing_data.keys():
            existing_data[key].update(new_data[key])
        else:
            existing_data[key] = new_data[key]
    return existing_data

In [3]:
def prepare_matrix_data(json_object):
    lo_data = {
        'knowing': {
            'multiple_choice': {
                'num_questions': 0,
                'score': 0
            },
            'free_form': {
                'num_questions': 0,
                'score': 0
            }
        },
        'understanding': {
            'multiple_choice': {
                'num_questions': 0,
                'score': 0
            },
            'free_form': {
                'num_questions': 0,
                'score': 0
            }
        },
        'application': {
            'multiple_choice': {
                'num_questions': 0,
                'score': 0
            },
            'free_form': {
                'num_questions': 0,
                'score': 0
            }
        },
        'adv_application': {
            'multiple_choice': {
                'num_questions': 0,
                'score': 0
            },
            'free_form': {
                'num_questions': 0,
                'score': 0
            }
        }
    }

    question_types = {
        'Trắc nghiệm': 'multiple_choice',
        'Tự luận': 'free_form'
    }

    knowledge_levels = {
        'Nhận biết': 'knowing',
        'Thông hiểu': 'understanding',
        'Vận dụng': 'application',
        'Vận dụng cao': 'adv_application'
    }

    matrix_data = {}

    for item in json_object:
        chapter = item["Chương hoặc Chủ đề"]
        learning_objective = item["Nội dung hoặc Đơn vị Kiến thức"]
        mastery_level = item["Mức độ Nhận thức"]
        question_type = item["Loại câu hỏi"]
        question_num = item["Số câu hỏi"]
        total_score = item["Tổng số điểm"]
        
        lo_question = lo_data

        lo_question[knowledge_levels[mastery_level]][question_types[question_type]]['num_questions'] = int(question_num)
        lo_question[knowledge_levels[mastery_level]][question_types[question_type]]['score'] = int(total_score)

        if chapter not in matrix_data.keys():
            matrix_data[chapter] = {learning_objective: lo_question}
        else:
            if learning_objective not in matrix_data[chapter].keys():
                matrix_data[chapter][learning_objective] = lo_question
            else:
                matrix_data[chapter][learning_objective] = merge_learning_objects(matrix_data[chapter][learning_objective], lo_question)

    return matrix_data

In [4]:
def percentage_calculation(matrix_data):
    total = {
            'knowing': {
                'multiple_choice': {
                    'total_questions': 0,
                    'total_score': 0
                },
                'free_form': {
                    'total_questions': 0,
                    'total_score': 0
                }
            },
            'understanding': {
                'multiple_choice': {
                    'total_questions': 0,
                    'total_score': 0
                },
                'free_form': {
                    'total_questions': 0,
                    'total_score': 0
                }
            },
            'application': {
                'multiple_choice': {
                    'total_questions': 0,
                    'total_score': 0
                },
                'free_form': {
                    'total_questions': 0,
                    'total_score': 0
                }
            },
            'adv_application': {
                'multiple_choice': {
                    'total_questions': 0,
                    'total_score': 0
                },
                'free_form': {
                    'total_questions': 0,
                    'total_score': 0
                }
            }
        }

    for chapter in matrix_data.keys():
        for learning_objective in matrix_data[chapter].keys():
            for kl in matrix_data[chapter][learning_objective].keys():
                for question in matrix_data[chapter][learning_objective][kl].keys():
                    total[kl][question]['total_questions'] += matrix_data[chapter][learning_objective][kl][question]['num_questions']
                    total[kl][question]['total_score'] += matrix_data[chapter][learning_objective][kl][question]['score']

    return total

In [5]:
def generating_exam_matrix(matrix_json_obj):
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1" />
        <title></title>
        <style>
        html {
            font-size: 10px;
        }
        body {
            font-family: Georgia, Arial, Helvetica, sans-serif;
            font-size: 1.6rem;
        }
        h1 {
            font-size: 2rem;
            text-align: center;
            padding: 1rem 0;
        }
        table {
            width: 100%;
            border-style: solid;
            border-color:#000;
            border-width: 1px 0 0 1px;
            border-spacing: 0px;
        }
        td, th {
            border: 1px solid #000;
            border-width: 0 1px 1px 0;
            padding: .25rem .5rem;
        }
        thead th {
            background-color: #f0f0f0;
        }
        tbody td:first-child {
            text-align: left;
        }
        tfoot td {
            text-align: center;
            font-weight: bold;
            background-color: #f0f0f0;
        }

        .columnDataNumber {
            text-align: right;
        }
        .borderBottomDashed {
            border-bottom: 1px dashed #000;
        }
        </style>
        <meta name="description" content="" />
    </head>

    <body>
        <h1>
        MA TRẬN KIỂM TRA GIỮA HỌC KỲ I - NĂM HỌC 2023 - 2024
        <br>
        MÔN TOÁN 9 - THỜI GIAN LÀM BÀI 90 PHÚT
        </h1>

        <!-- NOTE: The table has 14 columns -->
        <table>
        <thead>
            <tr>
            <th rowspan="3">TT</th>
            <th rowspan="3">Nội dung Kiến thức</th>
            <th rowspan="3">Đơn vị Kiến thức</th>
            <th colspan="8">Mức độ Nhận thức</th>
            <!-- <th></th>
            <th></th>
            <th></th>
            <th></th>
            <th></th>
            <th></th>
            <th></th> -->
            <th colspan="2">Tổng</th>
            <!-- <th></th> -->
            <th rowspan="3" class="columnDataNumber">Tổng điểm</th>
            </tr>
            <tr>
            <!-- <th rowspan="3">TT</th>
            <th rowspan="3">Nội dung Kiến thức</th>
            <th rowspan="3">Đơn vị Kiến thức</th> -->
            <th colspan="2">Nhận biết</th>
            <!--<th></th> -->
            <th colspan="2">Thông hiểu</th>
            <!--<th></th> -->
            <th colspan="2">Vận dụng thấp</th>
            <!--<th></th> -->
            <th colspan="2">Vận dụng cao</th>
            <!--<th></th> -->
            <th colspan="2">Số câu hỏi</th>
            <!--<th></th> -->
            <!-- <th rowspan="3">Tổng điểm</th> -->
            </tr>
            <tr>
            <!-- <th rowspan="3">TT</th>
            <th rowspan="3">Nội dung Kiến thức</th>
            <th rowspan="3">Đơn vị Kiến thức</th> -->
            <th>TN</th>
            <th>TL</th>
            <th>TN</th>
            <th>TL</th>
            <th>TN</th>
            <th>TL</th>
            <th>TN</th>
            <th>TL</th>
            <th>TN</th>
            <th>TL</th>
            <!-- <th rowspan="3">Tổng điểm</th> -->
            </tr>
        </thead>
                <tbody>
            """
    matrix_data = prepare_matrix_data(data)
    
    for seq, chapter in enumerate(matrix_data.keys()):
        html_content += f"""
        <tr>
    <td rowspan="{len(matrix_data[chapter]) * 2}">{seq + 1}</td>
    <td rowspan="{len(matrix_data[chapter]) * 2}">{chapter}</td>
        """
        for lo in matrix_data[chapter].keys():
            html_content += f"""
            <td rowspan="2">{lo}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['knowing']['multiple_choice']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['knowing']['free_form']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['understanding']['multiple_choice']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['understanding']['free_form']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['application']['multiple_choice']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['application']['free_form']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['adv_application']['multiple_choice']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['adv_application']['free_form']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{sum([matrix_data[chapter][lo][level]['multiple_choice']['num_questions'] for level in ['knowing', 'understanding', 'application', 'adv_application']])}</td>
            <td class="columnDataNumber borderBottomDashed">{sum([matrix_data[chapter][lo][level]['free_form']['num_questions'] for level in ['knowing', 'understanding', 'application', 'adv_application']])}</td>
            <td class="columnDataNumber borderBottomDashed"></td>
            </tr>
            <tr>
            <!-- <td rowspan="4">1</td>
            <td rowspan="4">Căn bậc hai, căn bậc ba</td>
            <td rowspan="2">Căn bậc hai</td> -->
            <td class="columnDataNumber">{matrix_data[chapter][lo]['knowing']['multiple_choice']['score']}</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['knowing']['free_form']['score']}</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['understanding']['multiple_choice']['score']}</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['understanding']['free_form']['score']}</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['application']['multiple_choice']['score']}</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['application']['free_form']['score']}</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['adv_application']['multiple_choice']['score']}</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['adv_application']['free_form']['score']}</td>
            <td class="columnDataNumber"></td>
            <td class="columnDataNumber"></td>
            <td class="columnDataNumber">{sum([matrix_data[chapter][lo][level]['multiple_choice']['score'] for level in ['knowing', 'understanding', 'application', 'adv_application']]) +  sum([matrix_data[chapter][lo][level]['free_form']['score'] for level in ['knowing', 'understanding', 'application', 'adv_application']])} </td>
            </tr>
            """
    total = percentage_calculation(matrix_data)

    total_score = sum([total[kl]['multiple_choice']['total_score'] for kl in ['knowing', 'understanding', 'application', 'adv_application']]) + sum([total[kl]['free_form']['total_score'] for kl in ['knowing', 'understanding', 'application', 'adv_application']])
    knowing_score = sum([total['knowing'][q]['total_score'] for q in ['multiple_choice', 'free_form']])
    understanding_score = sum([total['understanding'][q]['total_score'] for q in ['multiple_choice', 'free_form']])
    application_score = sum([total['application'][q]['total_score'] for q in ['multiple_choice', 'free_form']])
    adv_application_score = sum([total['adv_application'][q]['total_score'] for q in ['multiple_choice', 'free_form']])

    html_content += f"""
            </tbody>
            <tfoot>
            <tr>
            <td colspan="3">Tổng</td>
            <!-- <td></td>
            <td></td> -->
            <td class="columnDataNumber">{total['knowing']['multiple_choice']['total_questions']}</td>
            <td class="columnDataNumber">{total['knowing']['free_form']['total_questions']}</td>
            <td class="columnDataNumber">{total['understanding']['multiple_choice']['total_questions']}</td>
            <td class="columnDataNumber">{total['understanding']['free_form']['total_questions']}</td>
            <td class="columnDataNumber">{total['application']['multiple_choice']['total_questions']}</td>
            <td class="columnDataNumber">{total['application']['free_form']['total_questions']}</td>
            <td class="columnDataNumber">{total['adv_application']['multiple_choice']['total_questions']}</td>
            <td class="columnDataNumber">{total['adv_application']['free_form']['total_questions']}</td>
            <td class="columnDataNumber">{sum([total[kl]['multiple_choice']['total_questions'] for kl in ['knowing', 'understanding', 'application', 'adv_application']])}</td>
            <td class="columnDataNumber">{sum([total[kl]['free_form']['total_questions'] for kl in ['knowing', 'understanding', 'application', 'adv_application']])}</td>
            <td class="columnDataNumber">{total_score}</td>
            </tr>
            <tr>
            <td colspan="3">Tỷ lệ %</td>
            <!-- <td></td>
            <td></td> -->
            <td colspan="2">{knowing_score/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <td colspan="2">{understanding_score/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <td colspan="2">{application_score/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <td colspan="2">{adv_application_score/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <td colspan="2">{(knowing_score + understanding_score + application_score + adv_application_score)/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <td class="columnDataNumber"></td>
            </tr>
            <tr>
            <td colspan="3">Tỷ lệ chung %</td>
            <!-- <td></td>
            <td></td> -->
            <td colspan="4">{(knowing_score + understanding_score)/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <!-- <td></td> -->
            <!-- <td></td> -->
            <td colspan="4">{(application_score + adv_application_score)/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <!-- <td></td> -->
            <!-- <td></td> -->
            <td colspan="2">{(knowing_score + understanding_score + application_score + adv_application_score)/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <td class="columnDataNumber"></td>
            </tr>
        </tfoot>
    </body>
    </html>
    """
    return html_content

Hướng dẫn:
Chọn đường dẫn tới file matrix_json_file chứa dữ liệu về ma trận đề bài và file matrix_html_file là ma trận đã được tạo ra ở định dạng HTML

In [8]:
matrix_json_file = './matrix.json'
matrix_html_file = './matrix.html'


In [9]:
import json
with open(matrix_json_file, 'r', encoding='utf-8') as file:
    data = json.load(file)
    html = generating_exam_matrix(data)
    with open(matrix_html_file, "w", encoding="utf-8") as matrix:
        matrix.write(html)

Tạo đề thi

In [10]:
def compile_questions(question_sets):
    list_of_questions = ""
    list_of_tl = ""

    for key, value in question_sets.items():
        question_items = value.get('Câu hỏi').replace(' \\ ', ' \\\ ').split('\n')
        answer = ""
        for ans in question_items[1:]:
            if ans:
                answer += fr"""<li><div class="marginLeft2">{ans}</div></li>"""

            question = fr"""<!-- A question: -->
                <article class="QuestionItem marginBottom2">
                <h3>
                    <strong><u>Câu {key.replace('BÀI ', '')}</u></strong>
                    <span>({value.get('Số điểm')} điểm)</span>
                    <span>{question_items[0]}</span>

                </h3>
                <ul class="QuestionItem-options columnNumbers4">
                {answer}
                </ul>
                </article>
                <!-- /A question -->
                """.replace('\x08', '\\b')

        if value.get("Thông tin Nội bộ (học sinh không thấy)").get("Loại câu hỏi") == "Trắc nghiệm":
            list_of_questions += question
        elif value.get("Thông tin Nội bộ (học sinh không thấy)").get("Loại câu hỏi") == "Tự luận":
            list_of_tl += question

    return list_of_questions, list_of_tl

<>:6: SyntaxWarning: invalid escape sequence '\ '
<>:6: SyntaxWarning: invalid escape sequence '\ '
/var/folders/hm/z817kn_x6cx0w_cdvvwb_kc40000gn/T/ipykernel_2411/4082425144.py:6: SyntaxWarning: invalid escape sequence '\ '
  question_items = value.get('Câu hỏi').replace(' \\ ', ' \\\ ').split('\n')


In [11]:
def merge_all_sections(list_of_questions, list_of_tl):
    TEST_TMPL = """
    <!DOCTYPE html>
    <html>
        <head>
        <meta charset="utf-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1" />
        <title></title>
        <script type="text/javascript" id="MathJax-script" async
            src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-chtml.js">
        </script>
        <style>
            * {
                box-sizing: border-box;
            }
            html {
            font-size: 10px;
            }
            body {
            font-family: 'Times New Roman', Times, serif;
            font-size: 1.6rem;
            }
            h1 {
            font-size: 2rem;
            text-align: center;
            margin: 0 0 .5rem;
            }
            h2, h3 {
            font-size: 1.6rem;
            font-weight: normal;
            margin-bottom: 0;
            }
            .container {
            width: 100%;
            max-width: 80rem;
            margin: 0 auto;
            padding: 2.4rem 1.6rem;
            }
            .marginLeft1 {
            margin-left: .25rem;
            }
            .marginLeft2 {
            margin-left: .5rem;
            }
            .marginBottom1 {
            margin-bottom: .5rem;
            }
            .marginBottom1 {
            margin-bottom: 1rem;
            }
            .marginBottom2 {
            margin-bottom: 1.5rem;
            }
            .marginBottom3 {
            margin-bottom: 3rem;
            }

            footer {
            text-align: center;
            }


            table {
            width: 100%;
            border-width: 0;
            border-spacing: 0px;
            }
            td {
            padding: .25rem .5rem;
            text-align: center;
            }

            .QuestionItem {}
            .QuestionItem-options {
            list-style-type: none;
            padding-left: 4.5rem;
            margin: 0;
            display: flex;
            flex-wrap: wrap;
            }
            .QuestionItem-options li {
            padding-right: 1.5rem;
            margin-top: 1rem;
            display: flex;
            }
            .QuestionItem-options li.isCorrectAnswer > strong {
            color: red;
            }

            .QuestionItem-options.columnNumbers1 li {
            width: 100%;
            }
            .QuestionItem-options.columnNumbers2 li {
            width: 50%;
            }
            .QuestionItem-options.columnNumbers3 li {
            width: 33.33%;
            }
            .QuestionItem-options.columnNumbers4 li {
            width: 25%;
            }


        </style>
        <meta name="description" content="" />
        </head>"""

    title = f"""<body>
        <div class="container">
            <header class="marginBottom3">
                <table>
                    <tr>
                        <td>
                            <div>{province}</div>
                            <div><strong>{school}</strong></div>
                            <br>
                            <div><strong>Đề chính thức</strong></div>
                            <div><em>(Đề gồm 01 trang)</em></div>
                        </td>
                        <td>
                            <h1>ĐỀ KIỂM TRA GIỮA KÌ HỌC KÌ II</h1>
                            <div>
                                <strong>Năm học: 2023 - 2024</strong><br>
                                <strong>Môn: Toán</strong><br>
                                <strong>Lớp: 9</strong><br>
                                <strong>Thời gian: 90 phút</strong>&nbsp;<em>(Không kể thời gian giao đề)</em>
                            </div>
                        </td>
                    </tr>
                </table>
            </header>
    """

    part_1 = fr"""
            <main>
                <!-- PART 1: TRẮC NGHIỆM -->
                <section class="marginBottom3">
                    <h2>
                        <strong><u>I. Trắc nghiệm:</u></strong>
                        <em class="marginLeft1">(4,0 điểm).</em>
                        <strong class="marginLeft1"><em>Chọn câu trả lời đúng nhất trong các câu sau:</em></strong>
                    </h2>

                    {list_of_questions}

                </section>"""
    part_2 = fr"""
                <!-- PART 2: TỰ LUẬN -->
                <section class="marginBottom3">
                    <h2>
                        <strong><u>II. Tự luận:</u></strong>
                        <em class="marginLeft1">(6,0 điểm).</em>
                    </h2>

                    {list_of_tl}

                </section>
            </main>"""

    footer = """
            <footer>
                <div><em>.............................Hết................................</em></div>
                <div><em>Cán bộ coi thi không giải thích gì thêm!</em></div>
            </footer>
        </div>
    </html>"""
    return TEST_TMPL+title+part_1+part_2+footer

In [12]:
def exam_generation(question_sets):
    list_of_questions, list_of_tl = compile_questions(question_sets)
    html_content = merge_all_sections(list_of_questions, list_of_tl)
    return html_content
    

In [18]:
exam_json_file = './exam.json'
exam_html_file = './exam.html'

In [19]:
province = "SỞ GD&ĐT HÀ GIANG"
school = "STEAM FOR VIETNAM"
# question_sets = {
#     "BÀI 1": {
#         "Câu hỏi": "Cho bất đẳng thức \(a \leq b\). Phát biểu nào sau đây đúng?\n\nA. \(a\) luôn nhỏ hơn \(b\)\nB. \(a\) có thể bằng \(b\)\nC. \(a\) luôn lớn hơn \(b\)\nD. \(a\) có thể lớn hơn hoặc bằng \(b\)\n",
#         "Số điểm": 4,
#         "Thông tin Nội bộ (học sinh không thấy)": {
#             "Chương hoặc Chủ đề": "Bất đẳng thức; Bất phương trình bậc nhất một ẩn",
#             "Nội dung hoặc Đơn vị Kiến Thức": "Bất Đẳng Thức",
#             "Mức độ Nhận thức": "Nhận biết",
#             "Loại câu hỏi": "Trắc nghiệm"
#         }
#     },
#     "BÀI 2": {
#         "Câu hỏi": "Nếu \(a \leq b\) và \(b \leq c\), phát biểu nào sau đây là đúng?\n\nA. \(a \geq c\)\nB. \(a = c\)\nC. \(a \leq c\)\nD. \(a\) và \(c\) không liên quan đến nhau\n",
#         "Số điểm": 8,
#         "Thông tin Nội bộ (học sinh không thấy)": {
#             "Chương hoặc Chủ đề": "Bất đẳng thức; Bất phương trình bậc nhất một ẩn",
#             "Nội dung hoặc Đơn vị Kiến Thức": "Bất Đẳng Thức",
#             "Mức độ Nhận thức": "Thông hiểu",
#             "Loại câu hỏi": "Trắc nghiệm"
#         }
#     },
#     "BÀI 3": {
#         "Câu hỏi": "Phương trình nào sau đây có nghiệm \(x = 2\)?\n\nA. \(x + 2 = 0\)\nB. \(2x - 4 = 0\)\nC. \(x^2 - 4 = 0\)\nD. \(3x + 1 = 7\)\n",
#         "Số điểm": 4,
#         "Thông tin Nội bộ (học sinh không thấy)": {
#             "Chương hoặc Chủ đề": "Phương trình và Hệ Phương trình",
#             "Nội dung hoặc Đơn vị Kiến Thức": "Phương Trình Bậc Nhất",
#             "Mức độ Nhận thức": "Nhận biết",
#             "Loại câu hỏi": "Trắc nghiệm"
#         }
#     },
#     "BÀI 4": {
#         "Câu hỏi": "Cho phương trình \(2x - 5 = 3\). Giá trị nào của \(x\) thỏa mãn phương trình?\n\nA. \(x = 1\)\nB. \(x = 4\)\nC. \(x = 3\)\nD. \(x = -1\)\n",
#         "Số điểm": 8,
#         "Thông tin Nội bộ (học sinh không thấy)": {
#             "Chương hoặc Chủ đề": "Phương trình và Hệ Phương trình",
#             "Nội dung hoặc Đơn vị Kiến Thức": "Phương Trình Bậc Nhất",
#             "Mức độ Nhận thức": "Thông hiểu",
#             "Loại câu hỏi": "Trắc nghiệm"
#         }
#     },
#     "BÀI 5": {
#         "Câu hỏi": "Số nào dưới đây là căn bậc hai của 81?\n\nA. 7\nB. 8\nC. 9\nD. 10\n",
#         "Số điểm": 4,
#         "Thông tin Nội bộ (học sinh không thấy)": {
#             "Chương hoặc Chủ đề": "Căn thức",
#             "Nội dung hoặc Đơn vị Kiến Thức": "Căn Bậc Hai",
#             "Mức độ Nhận Thức": "Nhận biết",
#             "Loại câu hỏi": "Trắc nghiệm"
#         }
#     },
#     "BÀI 6": {
#         "Câu hỏi": "Rút gọn biểu thức \(\sqrt{72}\).\n\nA. \(6\sqrt{2}\)\nB. \(4\sqrt{3}\)\nC. \(3\sqrt{6}\)\nD. \(2\sqrt{18}\)\n",
#         "Số điểm": 8,
#         "Thông tin Nội bộ (học sinh không thấy)": {
#             "Chương hoặc Chủ đề": "Căn thức",
#             "Nội dung hoặc Đơn vị Kiến Thức": "Căn Bậc Hai",
#             "Mức độ Nhận Thức": "Thông hiểu",
#             "Loại câu hỏi": "Trắc nghiệm"
#         }
#     },
#     "BÀI 7": {
#         "Câu hỏi": "Biểu thức nào dưới đây là căn bậc ba của 27?\n\nA. 3\nB. 9\nC. 27\nD. 81\n",
#         "Số điểm": 4,
#         "Thông tin Nội bộ (học sinh không thấy)": {
#             "Chương hoặc Chủ đề": "Căn thức",
#             "Nội dung hoặc Đơn vị Kiến Thức": "Căn Bậc Ba",
#             "Mức độ Nhận Thức": "Nhận biết",
#             "Loại câu hỏi": "Trắc nghiệm"
#         }
#     },
#     "BÀI 8": {
#         "Câu hỏi": "Rút gọn biểu thức căn bậc ba \(\sqrt[3]{64}\).\n\nA. 4\nB. 8\nC. 16\nD. 32\n",
#         "Số điểm": 8,
#         "Thông tin Nội bộ (học sinh không thấy)": {
#             "Chương hoặc Chủ đề": "Căn thức",
#             "Nội dung hoặc Đơn vị Kiến Thức": "Căn Bậc Ba",
#             "Mức độ Nhận Thức": "Thông hiểu",
#             "Loại câu hỏi": "Trắc nghiệm"
#         }
#     },
#     "BÀI 9": {
#         "Câu hỏi": "Giải hệ phương trình sau:\n\n\(\begin{cases} 2x + y = 5 \\ 3x - y = 4 \end{cases}\)\n",
#         "{Số điểm}": 8,
#         "Thông tin Nội bộ (học sinh không thấy)": {
#             "Chương hoặc Chủ đề": "Phương trình và Hệ Phương trình",
#             "Nội dung hoặc Đơn vị Kiến Thức": "Hệ Hai PT Bậc Nhất",
#             "Mức độ Nhận Thức": "Vận dụng",
#             "Loại câu hỏi": "Tự luận"
#         }
#     },
#     "BÀI 10": {
#         "Câu hỏi": "Giải bất phương trình và biểu diễn tập nghiệm trên trục số:\n\n\(2x - 3 > 1\)\n",
#         "Số điểm": 8,
#         "Thông tin Nội bộ (học sinh không thấy)": {
#             "Chương hoặc Chủ đề": "Bất đẳng thức; Bất phương trình bậc nhất một ẩn",
#             "Nội dung hoặc Đơn vị Kiến Thức": "BPT Bậc Nhất Một Ẩn",
#             "Mức độ Nhận Thức": "Vận dụng",
#             "Loại câu hỏi": "Tự luận"
#         }
#     }
# }

In [20]:
# html = exam_generation(question_sets)
# with open(exam_html_file, "w", encoding="utf-8") as exam:
#     exam.write(html)

In [22]:
with open(exam_json_file, 'r', encoding='utf-8') as file:
    question_sets = json.load(file)
    html = exam_generation(question_sets)
    with open(exam_html_file, "w", encoding="utf-8") as exam:
        exam.write(html)